In [1]:
import numpy as np
import pandas as pd
import re

In [2]:
# load in all relevant data packages
directionality_data = pd.read_csv("Zmays_493_RefGen_V4.gene_exons.gff3", sep='\t')
expression_data = pd.read_csv("942_FPKM_B73_genes_w_feature.txt", sep='\t')

C:\Users\15099\Downloads\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)
C:\Users\15099\Downloads\Programs\Anaconda\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (1) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [4]:
# See format of directionality data
# Note: Had to reformat the directionality data from phytozome, removed the header and added my own header

# Uncomment below line to view
# directionality_data.head()

In [5]:
# See format of expression data

# Uncomment below line to view
# expression_data.head()

In [6]:
# remove all data that aren't genes
expression_data = expression_data[expression_data['feature_type'] == 'gene']
directionality_data = directionality_data[directionality_data['Region'] == 'gene']

# remove all data not mapped to a chromosome
expression_data = expression_data[pd.to_numeric(expression_data['chromosome'], errors='coerce').notnull()]

In [7]:
# Isolate just the gene name from phytozome ID name (formatting data)
# ID=Zm00001d027230.RefGen_V4;Name=Zm00001d027230 --> Zm00001d027230
ID_name = []
for j in directionality_data.ID:
    x = re.search('Name=(.+)', j)
    ID_name.append(x.group(1))
directionality_data.ID = ID_name

In [8]:
# Merge data sets into one file
merged_data = pd.merge(expression_data, directionality_data, left_on='gene', right_on='ID', how='inner')

In [9]:
# Calculate variance between all of the different maize lines
merged_data['Variance'] = merged_data.iloc[:,5:947].var(axis=1)

In [10]:
finalized = merged_data[['gene','chromosome', 'position_left', 'position_right', 'Direction', 'Variance']]

In [11]:
# View data layout
finalized.head()

,gene,chromosome,position_left,position_right,Direction,Variance
0,Zm00001d012719,8,179164454,179168169,-,8.863634
1,Zm00001d024742,10,85863323,85863746,+,0.002510
2,Zm00001d025653,10,125107973,125113328,-,7.309110
3,Zm00001d036003,6,65322472,65323171,+,23.779891
4,Zm00001d007933,2,243388293,243389628,+,0.061605


In [12]:
# Transfer
finalized.to_csv("condensed_subset_Davis.csv")